# Generate stores & customers for the demo

## Setup the environment

### Install packages

In [1]:
!pip install faker
!pip install pymorton
!pip install v3io
!pip install v3io_frames
!pip install boto3

    100% |████████████████████████████████| 880kB 19.5MB/s 
    100% |████████████████████████████████| 81kB 22.5MB/s 
    100% |████████████████████████████████| 61kB 21.3MB/s 
    100% |████████████████████████████████| 194kB 19.7MB/s 
  Running setup.py bdist_wheel for v3io ... done
  Stored in directory: /igz/.cache/pip/wheels/16/06/30/71478da3b7ae64df011a3466bcea2653a35ba3c5aa43ca1331
  Running setup.py bdist_wheel for ujson ... done
  Stored in directory: /igz/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built v3io ujson
nuclio-jupyter 0.7.2 has requirement tornado<6,>=5, but you'll have tornado 6.0.2 which is incompatible.
  Found existing installation: requests 2.22.0
    Uninstalling requests-2.22.0:
      Successfully uninstalled requests-2.22.0


### Define environment variables

In [5]:
# Stores
NUMBER_OF_STORES=500
STORES_TABLE='stores'
ACCURACY=20
BASE_ACCURACY=14

# Customers
NUMBER_OF_CUSTOMERS=3000
CUSTOMERS_TABLE='customers'
os.environ['CUSTOMERS_DATA_FILE_DIR'] = os.path.join(os.getcwd(), 'data')
os.environ['CUSTOMERS_DATA_FILE_PATH'] = os.path.join(os.environ['CUSTOMERS_DATA_FILE_DIR'], 'data_train_n.csv')
!mkdir -p ${CUSTOMERS_DATA_FILE_DIR}

In [6]:
files = ['data_train_n.csv']

bucket = 'iguazio-sample-data'
config= botocore.config.Config(signature_version=botocore.UNSIGNED)
s3 = boto3.client('s3', config=config)
for file in files:
   try:
        with open(os.environ['CUSTOMERS_DATA_FILE_PATH'], 'wb') as f:
           s3.download_fileobj(bucket, file, f)
   except botocore.exceptions.ClientError as e:
       if e.response['Error']['Code'] == "404":
           print("The object does not exist.")
       else:
           raise

In [9]:
!ls -lah ${CUSTOMERS_DATA_FILE_DIR}

total 0
-rw-r--r-- 1 50 nogroup 400M Jul  7 10:12 data_train_n.csv


### imports

In [1]:
import logging
import os
import pickle
import random
import itertools
import boto3
import botocore

# Demo
import pandas as pd
import pymorton as pm
from faker import Faker
from faker.providers import BaseProvider

# DB
import v3io as v3c
import v3io_frames as v3f

### Open DB client

In [10]:
client = v3f.Client('framesd:8081')

## Generate Stores

### Location provider

In [11]:
class LocationProvider(BaseProvider):
    '''
    Creates locations within base_location

    Uses QuadTree for Geohashing
        @{http://tech.taskrabbit.com/blog/2015/06/09/elasticsearch-geohash-vs-geotree/}
        @{http://mapzen.github.io/leaflet-spatial-prefix-tree/}
        @{http://blog.notdot.net/2009/11/Damn-Cool-Algorithms-Spatial-indexing-with-Quadtrees-and-Hilbert-Curves}
    '''    
    def location(self, location_base: str, base_acc=10, acc=20):
        coordinates = location_base[:base_acc]
        for i in range(acc-len(coordinates)):
            coordinates += str(random.randint(0, 3))
        return coordinates

Add the location provider as a faker provider

In [12]:
faker = Faker()
faker.add_provider(LocationProvider)

### Define scenario environment (locations)

In [13]:
london_city = (51.514926, -0.089580)
london_city_south = (51.501593, -0.094942)
london_west = (51.512309, -0.128966)
london_south_west = (51.495022, -0.162268)

london_coordinates = []
london_coordinates.append(london_city)
london_coordinates.append(london_city_south)
london_coordinates.append(london_west)
london_coordinates.append(london_south_west)

london_coordinates_qt = list(map(lambda cooridnate: pm.interleave_latlng(*cooridnate), london_coordinates))
london_coordinates_qt

['03311311313011311011000321002320',
 '03311311311233323013031101320003',
 '03311311313010023000032330133111',
 '03311311311222300331010333220231']

### Define Store class

In [14]:
class Store(object):
    def __init__(self, store_id: int, name: str, location: str):
        self.store_id = store_id if store_id is not None else random.randint(1, int(os.genev('NUMBER_OF_STORES', 500)))
        self.location = location
        (self.lat, self.long) = pm.deinterleave_latlng(location)
        self.store_name = name
        self.customers = 0
    
    def json(self):
        json_store = {
            'id': self.store_id,
            'name': self.store_name,
            'location': self.location,
            'latitude': self.lat,
            'longitude': self.long,
            'customers': self.customers
        }
        return json_store

### Create stores

In [15]:
def create_stores(faker, number_of_stores: int=500):
    remaining = number_of_stores
    stores = []
    while remaining > 0:
        # Create stores
        stores_batch = [Store(faker.msisdn(), faker.company(), faker.location(random.choice(london_coordinates_qt), BASE_ACCURACY, ACCURACY)).json() 
              for i in range(remaining)]
        
        # Count for duplicate locations
        locations = list(map(lambda store: store['location'], stores_batch))
        locations = list(dict.fromkeys(locations))
        remaining -= len(locations)
        
        # Update stores list
        stores.append(stores_batch)

    stores = list(itertools.chain.from_iterable(stores))
    stores = pd.DataFrame.from_records(stores)
    stores = stores.set_index(['location'])
    stores = stores[~stores.index.duplicated(keep='first')]
    return stores

In [17]:
client.delete('kv', STORES_TABLE, if_missing=1)

DeleteError: error in _delete: <_Rendezvous of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "can't delete: GetItems failed during recursive delete of 'stores/'.: Expected a 2xx response status code: HTTP/1.1 404 Not Found
Server: nginx
Date: Sun, 07 Jul 2019 10:13:42 GMT
Content-Type: application/json
Content-Length: 66
Connection: keep-alive

{
	"ErrorCode": -2,
	"ErrorMessage": "No such file or directory"
}"
	debug_error_string = "{"created":"@1562494423.090667760","description":"Error received from peer","file":"src/core/lib/surface/call.cc","file_line":1017,"grpc_message":"can't delete: GetItems failed during recursive delete of 'stores/'.: Expected a 2xx response status code: HTTP/1.1 404 Not Found\r\nServer: nginx\r\nDate: Sun, 07 Jul 2019 10:13:42 GMT\r\nContent-Type: application/json\r\nContent-Length: 66\r\nConnection: keep-alive\r\n\r\n{\n\t"ErrorCode": -2,\n\t"ErrorMessage": "No such file or directory"\n}","grpc_status":2}"
>

In [18]:
stores = create_stores(faker, NUMBER_OF_STORES)
stores.head(5)

,customers,id,latitude,longitude,name
location,,,,,
03311311313011312333,0,3843250429733,51.520042,-0.090981,Jones Ltd
03311311311233131323,0,7560016407222,51.489830,-0.088921,"Kennedy, Burns and Wolfe"
03311311313011301210,0,4389150341295,51.516266,-0.095444,Lang LLC
03311311311222212003,0,0718232756938,51.496010,-0.169945,"Williams, Hobbs and Gardner"
03311311311233203201,0,6940166429223,51.497040,-0.106773,Payne PLC


### Upload to DB 

In [19]:
# client.delete('kv', STORES_TABLE)
client.write('kv', STORES_TABLE, stores)

In [20]:
client.read('kv', STORES_TABLE).head(5)

,name,customers,id,latitude,longitude
location,,,,,
03311311311222022002,Clark PLC,0.0,5222948143156,51.490517,-0.175781
03311311313010220203,Howard Group,0.0,4098356409669,51.522102,-0.131493
03311311313011202321,Marshall-White,0.0,6065424030842,51.519699,-0.108147
03311311313011130232,"Johnson, Gonzales and Lee",0.0,5913255917578,51.511803,-0.092697
03311311313011223103,Steele PLC,0.0,6309532093786,51.523476,-0.105400


## Generate customers

In [21]:
class Customer(object):
    def __init__(self, id=0, location='0', products=""):
        self.id = id
        self.location = location
        self.products = products
        
    def json(self):
        json_customer = {
            'id': self.id,
            'location': self.location,
            'products': self.products
        }
        return json_customer

In [22]:
def create_customers(customers_data: pd.DataFrame, number_of_customers: int):
    available_customer = list(customers_data.user_id)
    customers = [Customer(id=i,
                          products=customers_data.loc[customers_data['user_id'] == random.choice(available_customer)]\
                                    .drop(['user_id', 'Unnamed: 0'], axis=1)\
                                    .to_json()).json() for i in range(number_of_customers)]
    customers = pd.DataFrame.from_records(customers)
    customers.set_index(['id'])
    return customers

In [26]:
# Load customers seed data
customers_data = pd.read_csv(os.environ['CUSTOMERS_DATA_FILE_PATH'])

# Create actual customers
customers = create_customers(customers_data, NUMBER_OF_CUSTOMERS)
customers.head(5)

,id,location,products
0,0,0,"{""user_product_reordered_ratio"":{""324119"":1.0,..."
1,1,0,"{""user_product_reordered_ratio"":{""299728"":1.0,..."
2,2,0,"{""user_product_reordered_ratio"":{""296781"":1.0,..."
3,3,0,"{""user_product_reordered_ratio"":{""480528"":1.0}..."
4,4,0,"{""user_product_reordered_ratio"":{""9664"":1.0,""3..."


## Upload to DB

In [ ]:
client.delete('kv', CUSTOMERS_TABLE, if_missing=1)
client.write('kv', CUSTOMERS_TABLE, dfs=customers)

In [45]:
print(f'Customers table sized: {client.read("kv", CUSTOMERS_TABLE).shape}')
client.read('kv', CUSTOMERS_TABLE).head(5)

Customers table sized: (2990, 3)


,products,id,location
index,,,
1926,"{""user_product_reordered_ratio"":{""1246"":1.0,""2...",1926.0,0
1517,"{""user_product_reordered_ratio"":{""178158"":1.0,...",1517.0,0
2183,"{""user_product_reordered_ratio"":{""9944"":1.0,""3...",2183.0,0
1650,"{""user_product_reordered_ratio"":{""67635"":1.0,""...",1650.0,0
1472,"{""user_product_reordered_ratio"":{""5811"":1.0,""2...",1472.0,0


## Generate predictions

In [47]:
client.delete('tsdb', 'predictions', if_missing=1)
client.create('tsdb', 'predictions', attrs={'rate': '1/s'}, if_exists=1)

CreateError: error in _create: <_Rendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Socket closed"
	debug_error_string = "{"created":"@1562494939.937252206","description":"Error received from peer","file":"src/core/lib/surface/call.cc","file_line":1017,"grpc_message":"Socket closed","grpc_status":14}"
>